In [2]:
import tensorflow as tf
import os

def csvread(filelist):
    """
    读取CSV文件
    :param filelist:文件路径+名字的列表
    :return:返回读取的内容
    """
    # 1、构造文件的队列
    file_queue = tf.train.string_input_producer(filelist)
    # 2、构造CSV阅读器读取队列数据（按照一行读取
    reader = tf.TextLineReader()
    key, value = reader.read(file_queue)
    # 3、对每行内容进行解码
    # 其中参数record_defaults：指定每个样本每列的类型及指定默认值，如records = [["None"], [4.0]]
    records = [["None"], ["None"]]
    example, label = tf.decode_csv(value, record_defaults=records)   # 对record解码，并指定每列的类型
    # 4、想要读取多个数据，批处理实现
    # 批处理的大小对队列、数据的数量没有影响，批处理大小只决定这批次取多少数据
    example_batch, label_batch = tf.train.batch([example, label], batch_size=9, num_threads=1, capacity=9)
    # print(example, label)
    # print(value)
    print(example_batch, label_batch)
    return example_batch, label_batch

def picread(filelist):
    """
    读取狗图片并转换成张量
    :param filelist: 文件路径+ 名字的列表
    :return: 每张图片的张量
    """
    # 1、构造文件队列
    file_queue = tf.train.string_input_producer(filelist)

    # 2、构造阅读器去读取图片内容（默认读取一张图片）
    reader = tf.WholeFileReader()
    key, value = reader.read(file_queue)
    print(value)

    # 3、对读取的图片数据进行解码
    image = tf.image.decode_jpeg(value)
    print(image)

    # 4、处理图片的大小（统一大小）
    image_resize = tf.image.resize_images(image, [200, 200])
    print(image_resize)

    # 注意：一定要把样本的形状固定 [200, 200, 3],在批处理的时候要求所有数据形状必须定义
    image_resize.set_shape([200, 200, 3])

    print(image_resize)

    # 5、进行批处理
    image_batch = tf.train.batch([image_resize], batch_size=20, num_threads=1, capacity=20)

    print(image_batch)

    return image_batch

# 定义cifar的数据等命令行参数
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string("cifar_dir", "./data/cifar10/cifar-10-batches-bin/", "文件的目录")
# tf.app.flags.DEFINE_string("cifar_tfrecords", "./tmp/cifar.tfrecords", "存进tfrecords的文件")

class CifarRead(object):
    """完成读取二进制文件， 写进tfrecords，读取tfrecords
    """
    def __init__(self, filelist):
        # 文件列表
        self.file_list = filelist

        # 定义读取的图片的一些属性
        self.height = 32
        self.width = 32
        self.channel = 3
        # 二进制文件每张图片的字节
        self.label_bytes = 1  #每张图片的标签占位1位字节
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes

    def read_and_decode(self):
        # 读取cifar文件 转换成张量

        # 1、构造文件队列
        file_queue = tf.train.string_input_producer(self.file_list)

        # 2、构造二进制文件读取器，读取内容, 每个样本的字节数
        reader = tf.FixedLengthRecordReader(self.bytes)

        key, value = reader.read(file_queue)

        # 3、解码内容, 二进制文件内容的解码
        label_image = tf.decode_raw(value, tf.uint8)

        print(label_image)

        # 4、分割出图片和标签数据，切出特征值和目标值
        label = tf.cast(tf.slice(label_image, [0], [self.label_bytes]), tf.int32)

        image = tf.slice(label_image, [self.label_bytes], [self.image_bytes])

        # 5、可以对图片的特征数据进行形状的改变 [3072] --> [32, 32, 3]
        image_reshape = tf.reshape(image, [self.height, self.width, self.channel])

        print(label, image_reshape)
        # 6、批处理数据
        image_batch, label_batch = tf.train.batch([image_reshape, label], batch_size=10, num_threads=1, capacity=10)

        print(image_batch, label_batch)
        return image_batch, label_batch


if __name__ == "__main__":
    # 1、找到文件，放入列表  路径+名字->放到列表当中
    file_name = os.listdir(FLAGS.cifar_dir)
    filelist = [os.path.join(FLAGS.cifar_dir, file) for file in file_name if file[-3:] == "bin"] #切片截取后缀

    # print(file_name)
    # print(file_list)
    cf = CifarRead(filelist)
    image_batch, label_batch = cf.read_and_decode()
    # 开启会话，运行结果
    with tf.Session() as sess:
        # 定义一个线程协调器
        coord = tf.train.Coordinator()
        # 开启读取文件的线程
        threads = tf.train.start_queue_runners(sess, coord=coord)
        # for i in range()
        # 打印读取的内容
        print(sess.run([image_batch, label_batch]))

        # 回收子线程
        coord.request_stop()
        coord.join(threads)


DuplicateFlagError: The flag 'cifar_dir' is defined twice. First from C:\Users\William Baker\Anaconda3\lib\site-packages\ipykernel_launcher.py, Second from C:\Users\William Baker\Anaconda3\lib\site-packages\ipykernel_launcher.py.  Description from first occurrence: 文件的目录